In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [77]:
import train_flow
importlib.reload(train_flow)

# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
"""
df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
#cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone
"""
# --------- #
# LEAVE ME  #
# --------- #
df_main = df_daily.copy()#[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope', 'past_return'])


# Run Train_Flow

In [ ]:
import train_flow
importlib.reload(train_flow)
from itertools import combinations

# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['run_separately'] #'run_together', 'run_separately'
train_flow.pi_handling_test(pi_handlings)
keys = list(feature_dict.keys())
two_combos = list(combinations(keys, 2))

master_results = []
master_perm = []

a = 'rsi_macd'
b = 'volu'
target_horizons = [5]#, 10, 20, 30]
train_years = [3]
pi_years = [1.5]
min_feats = [6]
days_assessed = 250
test_day = 5
feature_cols = feature_dict[a] + feature_dict[b]
list_name = f'{a}-{b}'
print(list_name)
groups = list_name.split("-")


# --------- #
# LEAVE ME  #
# --------- # 
models = {"xgboost-tr3-d6-c1-lr:.1": XGBClassifier(n_estimators=300, max_depth=6, colsample_bytree=1, min_child_weight=5, 
                                                    learning_rate=.1, random_state=42, n_jobs=-1)}
models = {"xgboost-3": XGBClassifier(n_estimators=300 random_state=42, n_jobs=-1)}
results_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                train_years, pi_years, min_feats, list_name, feature_dict, groups)

master_results.append(results_df)

"""
for a, b in two_combos:
# ---------- #
# UPDATE ME  #
# ---------- #
    target_horizons = [1, 2, 5, 10, 20, 30]
    train_years = [3]
    pi_years = [1.5]
    min_feats = [4]
    days_assessed = 250
    test_day = 5
    feature_cols = feature_dict[a] + feature_dict[b]
    list_name = f"{a}-{b}"
    print(list_name)
    groups = list_name.split("-")


    # --------- #
    # LEAVE ME  #
    # --------- # 
    models = {"xgboost-tr3-d6-c1-lr:.1": XGBClassifier(n_estimators=300, max_depth=6, colsample_bytree=1, min_child_weight=5, 
                                                        learning_rate=.1, random_state=42, n_jobs=-1)}
    models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
    results_df = train_flowv2.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                    train_years, pi_years, min_feats, list_name, feature_dict, groups)

    master_results.append(results_df)
    #master_perm.append(perm_df)



for a in keys:

    feature_cols = feature_dict[a] + feature_dict[b]
    list_name = f"{a}"
    print(list_name)
    groups = list_name.split("-")


    # --------- #
    # LEAVE ME  #
    # --------- # 
    #models = {"xgboost-3-f": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
    models = {"xgboost-1-10": XGBClassifier(n_estimators=100, max_depth=10, colsample_bytree=0.75, min_child_weight=10, random_state=42, n_jobs=-1)}
    models = {"xgboost-tr3-d6-c1-lr:.1": XGBClassifier(n_estimators=300, max_depth=6, colsample_bytree=1, min_child_weight=5, 
                                                        learning_rate=.1, random_state=42, n_jobs=-1)}
    models = {"xgboost-3": XGBClassifier(n_estimators=300, min_child_weight=5, random_state=42, n_jobs=-1)}
    results_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                    train_years, pi_years, min_feats, list_name, feature_dict, groups)

    master_results.append(results_df)
"""

# final master DataFrames
master_results_df = pd.concat(master_results, ignore_index=True)
#master_results_df.to_csv('h5_performance.csv', index=False)

for r in target_horizons:

    base_df = pd.read_csv(f"h{r}_performance_all.csv")
    cols = master_results_df.columns
    df_concat = pd.concat([base_df.drop_duplicates(), master_results_df[cols][master_results_df['horizon'] == r].drop_duplicates()], ignore_index=True)
    df_concat.to_csv(f"h{r}_performance_all2.csv", index=False)

import performance_flow
importlib.reload(performance_flow)

# ---------- #
# UPDATE ME  #
# ---------- #
results_file_name = "h5_performance_all2.csv" # Match prior cell saved as file name horizon_2_baseline_new
min_th = 0.51
cov_th = 0.75


# --------- #
# LEAVE ME  #
# --------- #
return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
#bucket_df = performance_flow.bucket_scores(df_daily, perf_df, returns, min_th)
cols = ['horizon', 'model', 'train_years', 'features', 'pi_size',
       'min_feats', 'pos_rate', 'bal_prec', 'pprec', 'nprec', 'pos_preds', 'neg_preds', 'composite']

composite_score[cols][composite_score['features'] == list_name]

rsi_macd-volu
Running for horizon 5 | run_separately
Run 1/50 | Train: 2023-02-24 → 2026-01-29 | Test: 2026-02-06 → 2026-02-12 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 4)
Run 2/50 | Train: 2023-02-16 → 2026-01-22 | Test: 2026-01-30 → 2026-02-05 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 4)
Run 3/50 | Train: 2023-02-09 → 2026-01-14 | Test: 2026-01-23 → 2026-01-29 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 4)
Run 4/50 | Train: 2023-02-02 → 2026-01-07 | Test: 2026-01-15 → 2026-01-22 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 4)
Run 5/50 | Train: 2023-01-26 → 2025-12-30 | Test: 2026-01-08 → 2026-01-14 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 4)
Run 6/50 | Train: 2023-01-19 → 2025-12-22 | Test: 2025-12-31 → 2026-01-07 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 4)
Run 7/50 | Train: 2023-01-11 → 2025-12-15 | Test: 2025-12-23 → 2025-12-30 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 4)
Run 8/50 | Train: 2023-01-04 → 2025-12-08 | Test: 2

,horizon,model,train_years,features,pi_size,min_feats,pos_rate,bal_prec,pprec,nprec,pos_preds,neg_preds,composite
30,5,xgboost-3,3,rsi_macd-volu,1.5,4,0.61,0.56,0.67,0.46,126.0,120.0,0.44
45,5,xgboost_tr1_d8,3,rsi_macd-volu,1.5,4,0.60,0.50,0.60,0.40,133.0,115.0,0.36


# Save most recent run

In [49]:
master_results_df = pd.concat(master_results, ignore_index=True)

for r in target_horizons:

    base_df = pd.read_csv(f"h{r}_performance_all2.csv")
    cols = master_results_df.columns
    df_concat = pd.concat([base_df.drop_duplicates(), master_results_df[cols][master_results_df['horizon'] == r].drop_duplicates()], ignore_index=True)
    df_concat.to_csv(f"h{r}_performance_all2.csv", index=False)

# Run Performance_Flow

In [74]:
import performance_flow
importlib.reload(performance_flow)

# ---------- #
# UPDATE ME  #
# ---------- #
results_file_name = "h5_performance_all.csv" # Match prior cell saved as file name horizon_2_baseline_new
min_th = 0.51
cov_th = 0.75


# --------- #
# LEAVE ME  #
# --------- #
return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
#bucket_df = performance_flow.bucket_scores(df_daily, perf_df, returns, min_th)
cols = ['horizon', 'model', 'train_years', 'features', 'pi_size',
       'min_feats', 'pos_rate', 'bal_prec', 'pprec', 'nprec', 'pos_preds', 'neg_preds', 'composite']

composite_score[cols]#[composite_score['features'] == 'ma_all']

1


,horizon,model,train_years,features,pi_size,min_feats,pos_rate,bal_prec,pprec,nprec,pos_preds,neg_preds,composite
30,5,xgboost-3,3,rsi_macd-volu,1.5,4,0.61,0.56,0.67,0.46,126.0,120.0,0.44
2,5,xgboost-3,3,ma_lag-ma_num,1.5,4,0.61,0.56,0.64,0.48,169.0,82.0,0.43
36,5,xgboost-3,3,volu-vix_skew,1.5,4,0.61,0.56,0.65,0.46,161.0,84.0,0.43
17,5,xgboost-3,3,ma_sma-ma_num,1.5,4,0.60,0.53,0.62,0.45,181.0,67.0,0.41
19,5,xgboost-3,3,ma_sma-volu,1.5,4,0.61,0.53,0.63,0.43,169.0,79.0,0.40
37,5,xgboost-3,3,volu-experimental_slope,1.5,4,0.60,0.53,0.63,0.43,131.0,119.0,0.40
12,5,xgboost-3,3,ma_rel-volu,1.5,4,0.60,0.53,0.62,0.43,157.0,91.0,0.40
3,5,xgboost-3,3,ma_lag-rsi_macd,1.5,4,0.62,0.53,0.65,0.41,119.0,131.0,0.40
24,5,xgboost-3,3,ma_num-rsi_macd,1.5,4,0.61,0.52,0.63,0.42,153.0,96.0,0.39
1,5,xgboost-3,3,ma_lag-ma_sma,1.5,4,0.61,0.52,0.63,0.41,140.0,111.0,0.39


In [31]:
perf_df['features'].unique()

array(['ma_lag-ma_rel', 'ma_lag-ma_sma', 'ma_lag-ma_num',
       'ma_lag-rsi_macd', 'ma_lag-volu', 'ma_lag-atr_adxvola',
       'ma_lag-vix_skew', 'ma_lag-experimental_slope',
       'ma_lag-past_return', 'ma_rel-ma_sma', 'ma_rel-ma_num',
       'ma_rel-rsi_macd', 'ma_rel-volu', 'ma_rel-atr_adxvola',
       'ma_rel-vix_skew', 'ma_rel-experimental_slope',
       'ma_rel-past_return', 'ma_sma-ma_num', 'ma_sma-rsi_macd',
       'ma_sma-volu', 'ma_sma-atr_adxvola', 'ma_sma-vix_skew',
       'ma_sma-experimental_slope', 'ma_sma-past_return',
       'ma_num-rsi_macd', 'ma_num-volu', 'ma_num-atr_adxvola',
       'ma_num-vix_skew', 'ma_num-experimental_slope',
       'ma_num-past_return', 'rsi_macd-volu', 'rsi_macd-atr_adxvola',
       'rsi_macd-vix_skew', 'rsi_macd-experimental_slope',
       'rsi_macd-past_return', 'volu-atr_adxvola', 'volu-vix_skew',
       'volu-experimental_slope', 'volu-past_return',
       'atr_adxvola-vix_skew', 'atr_adxvola-experimental_slope',
       'atr_adxvola-pas